In [2]:
import findspark
findspark.init()

In [3]:
from pyspark import SparkConf , SparkContext
conf = SparkConf().setMaster("local[4]").setAppName("BroadCastVariableObs")
sc = SparkContext(conf=conf).getOrCreate()

# product dosyasini okuyup urun id ve adini donduren fonk tanımlama

In [37]:
def read_products():
    products_text_wrapper = open("/home/taha/Downloads/products.txt","r",encoding = "utf-8")
    
    products = products_text_wrapper.readlines()
    product_id_name={}
    
    
    for line in products:
        
        if 'productCategoryId' not in line:
            prduct_id=int (line.split(',')[0])
            product_name = line.split(',')[2]
            
            product_id_name.update({prduct_id:product_name})
    
    return product_id_name

In [38]:
products = read_products()

In [39]:
broadcast_products = sc.broadcast(products)

In [40]:
broadcast_products.value.get(114)

"Nike Men's Fly Shorts 2.0"

# Order item okuma RDD olusturma

In [41]:
order_items_rdd2 = sc.textFile("/home/taha/Downloads/order_items.txt") 

In [42]:
order_items_RDD = order_items_rdd2.filter(lambda x :'orderItemOrderId' not in x)

In [43]:
order_items_RDD.take(5)

['1,1,957,1,299.98,299.98',
 '2,2,1073,1,199.99,199.99',
 '3,2,502,5,250.0,50.0',
 '4,2,403,1,129.99,129.99',
 '5,4,897,2,49.98,24.99']

# order items i pair RDD yapalım

In [58]:
def make_order_items_pair_rdd(line):
    order_item_product = int(line.split(',')[2])
    order_item_sub_total =float (line.split(',')[4])
    
    return (order_item_product,order_item_sub_total)

In [59]:
order_items_pair_Rdd=order_items_RDD.map(make_order_items_pair_rdd)

In [60]:
order_items_pair_Rdd.take(5)

[(957, 299.98), (1073, 199.99), (502, 250.0), (403, 129.99), (897, 49.98)]

In [61]:
rdd=order_items_pair_Rdd.reduceByKey(lambda x , y : x+y)



In [64]:
rdd=rdd.map(lambda x : (x[1],x[0]))

In [65]:
rdd.take(2)

[(1460400.0, 502), (1358106.5399999893, 1014)]

In [67]:
order_items_pair_Rdd=rdd.sortByKey(False)

In [68]:
order_items_pair_Rdd.take(10)

[(3232238.37999988, 1004),
 (2056937.1199998325, 365),
 (1897373.4999999227, 957),
 (1730026.9799999292, 191),
 (1460400.0, 502),
 (1438328.0799999484, 1073),
 (1358106.5399999893, 1014),
 (1329407.7299999283, 403),
 (596930.7300000004, 627),
 (39760.0, 565)]

In [69]:
order_items_pair_Rdd=order_items_pair_Rdd.map(lambda x : (x[1],x[0]))

In [70]:
order_items_pair_Rdd.take(10)

[(1004, 3232238.37999988),
 (365, 2056937.1199998325),
 (957, 1897373.4999999227),
 (191, 1730026.9799999292),
 (502, 1460400.0),
 (1073, 1438328.0799999484),
 (1014, 1358106.5399999893),
 (403, 1329407.7299999283),
 (627, 596930.7300000004),
 (565, 39760.0)]

 # order_items ile broadcast variable olan product i birlestirme

In [71]:
shorted_orders_with_product_name = order_items_pair_Rdd.map(lambda x :(broadcast_products.value.get(x[0]),x[1]))

In [72]:
shorted_orders_with_product_name.take(10)

[('Field & Stream Sportsman 16 Gun Fire Safe', 3232238.37999988),
 ('Perfect Fitness Perfect Rip Deck', 2056937.1199998325),
 ("Diamondback Women's Serene Classic Comfort Bi", 1897373.4999999227),
 ("Nike Men's Free 5.0+ Running Shoe", 1730026.9799999292),
 ("Nike Men's Dri-FIT Victory Golf Polo", 1460400.0),
 ('Pelican Sunstream 100 Kayak', 1438328.0799999484),
 ("O'Brien Men's Neoprene Life Vest", 1358106.5399999893),
 ("Nike Men's CJ Elite 2 TD Football Cleat", 1329407.7299999283),
 ("Under Armour Girls' Toddler Spine Surge Runni", 596930.7300000004),
 ('adidas Youth Germany Black/Red Away Match Soc', 39760.0)]